<a href="https://colab.research.google.com/github/vivi1231/0724/blob/master/0724titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
# from google.colab import files
# files.upload() #upload kaggle.json
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/

In [74]:
!kaggle competitions download -c titanic

test.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)
gender_submission.csv: Skipping, found more recently modified local copy (use --force to force download)


In [75]:
# from urllib.request import urlretrieve
# url = "https://github.com/Elwing-Chou/ml0716/raw/master/train.csv"
# urlretrieve(url, "train.csv")

In [76]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
test_df = pd.read_csv("test.csv", encoding="utf-8")

In [77]:
datas = pd.concat([train_df, test_df])
datas = datas.drop(["PassengerId", "Survived"], axis=1)
datas

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
413,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [78]:
# Step1. 補缺失值
s = datas.isna().sum()
# 篩選 series[[True....]]
s[s > 0]

Age          263
Fare           1
Cabin       1014
Embarked       2
dtype: int64

In [79]:
most = datas["Embarked"].value_counts().idxmax()
datas["Embarked"] = datas["Embarked"].fillna(most)

In [80]:
# 補數字類型[中位數]
med = datas.median().drop(["Pclass"])
d = datas.fillna(med)
d.isna().sum()

Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin       1014
Embarked       0
dtype: int64

In [81]:
# 先準備一下我要保留的list
def name(n):
    n = n.split(",")[-1].split(".")[0]
    return n.strip()
v = datas["Name"].apply(name).value_counts()
white = v.index[:4]

In [82]:
def name(n):
    n = n.split(",")[-1].split(".")[0]
    n = n.strip()
    if n in white:
        return n
    else:
        return None
datas["Name"] = datas["Name"].apply(name)
datas

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,Mr,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,Mrs,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,Miss,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,Mrs,female,35.0,1,0,113803,53.1000,C123,S
4,3,Mr,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
413,3,Mr,male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1,None,female,39.0,0,0,PC 17758,108.9000,C105,C
415,3,Mr,male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,3,Mr,male,NaN,0,0,359309,8.0500,NaN,S


In [83]:
v = datas["Ticket"].value_counts()
def ticket(t):
    global v
    return v[t]
datas["Ticket"] = datas["Ticket"].apply(ticket)

In [84]:
datas = datas.drop(["Cabin"], axis=1)

In [85]:
#如果想重製index: datas.reset_index(drop=True)
datas

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,3,Mr,male,22.0,1,0,1,7.2500,S
1,1,Mrs,female,38.0,1,0,2,71.2833,C
2,3,Miss,female,26.0,0,0,1,7.9250,S
3,1,Mrs,female,35.0,1,0,2,53.1000,S
4,3,Mr,male,35.0,0,0,1,8.0500,S
...,...,...,...,...,...,...,...,...,...
413,3,Mr,male,NaN,0,0,1,8.0500,S
414,1,None,female,39.0,0,0,3,108.9000,C
415,3,Mr,male,38.5,0,0,1,7.2500,S
416,3,Mr,male,NaN,0,0,1,8.0500,S


In [86]:
datas = pd.get_dummies(datas)
datas

,Pclass,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,1,7.2500,0,0,1,0,0,1,0,0,1
1,1,38.0,1,0,2,71.2833,0,0,0,1,1,0,1,0,0
2,3,26.0,0,0,1,7.9250,0,1,0,0,1,0,0,0,1
3,1,35.0,1,0,2,53.1000,0,0,0,1,1,0,0,0,1
4,3,35.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,3,NaN,0,0,1,8.0500,0,0,1,0,0,1,0,0,1
414,1,39.0,0,0,3,108.9000,0,0,0,0,1,0,1,0,0
415,3,38.5,0,0,1,7.2500,0,0,1,0,0,1,0,0,1
416,3,NaN,0,0,1,8.0500,0,0,1,0,0,1,0,0,1


In [87]:
# 取列: iloc(看實際是第幾列), loc(看列標籤)
# [第一列，第二列...] [:4]
x_train = datas.iloc[:len(train_df)]
x_predict = datas.iloc[len(train_df):]
y_train = train_df["Survived"]